# Using for iterator in Output Models

When generating structured outputs their may be the case where need a list of structured outputs based on the input given. The most common solution for this case is to run a task prompt multiple times for each input item. However, this can be inefficient and time consuming. To solve this problem, we can use the for iterator in the output model to generate a list of structured outputs based on the input given.


## Preparing the Input List

We will use our previous InputObject (Book) and generate a list of sequels by passing an BaseModel with "sequels: list[str]" as attribute.

In [1]:
from typing import Literal
from pydantic import BaseModel
import pydantic
from llmp.services.program import Program


class BookInput(BaseModel):
    book_title: str
    book_author: str
    release_year: int
    
class SequalOutput(BaseModel):
    sequels: list[str]
    

program = Program("gen sequels", BookInput, SequalOutput)

In [2]:
input_data={
    "book_title": "Harry Potter",
    "book_author": "J. K. Rowling",
    "release_year": 1997
}

output = program(input_data)

print(output.sequels)

['Harry Potter and the Chamber of Secrets', 'Harry Potter and the Prisoner of Azkaban', 'Harry Potter and the Goblet of Fire', 'Harry Potter and the Order of the Phoenix', 'Harry Potter and the Half-Blood Prince', 'Harry Potter and the Deathly Hallows']


## Iterating over the Input List

In [2]:
from pydantic import Field
from llmp.services.program import Program


class InputObject(BaseModel):
    book_list: list[str]
    
    
class OutputObject(BaseModel):
    book_data: list[BookInput] = Field(rule="for each $book in {book_list}")
    
    
program_iter = Program("gen sequels data", InputObject, OutputObject)   

In [4]:
result = program_iter({"book_list": output.sequels})

print(result.book_data)

[{'book_title': 'Harry Potter and the Chamber of Secrets', 'book_author': 'J.K. Rowling', 'release_year': 1998}, {'book_title': 'Harry Potter and the Prisoner of Azkaban', 'book_author': 'J.K. Rowling', 'release_year': 1999}, {'book_title': 'Harry Potter and the Goblet of Fire', 'book_author': 'J.K. Rowling', 'release_year': 2000}, {'book_title': 'Harry Potter and the Order of the Phoenix', 'book_author': 'J.K. Rowling', 'release_year': 2003}, {'book_title': 'Harry Potter and the Half-Blood Prince', 'book_author': 'J.K. Rowling', 'release_year': 2005}, {'book_title': 'Harry Potter and the Deathly Hallows', 'book_author': 'J.K. Rowling', 'release_year': 2007}]


In [10]:
import pprint

pprint.pprint(result.book_data)

[{'book_author': 'J.K. Rowling',
  'book_title': 'Harry Potter and the Chamber of Secrets',
  'release_year': 1998},
 {'book_author': 'J.K. Rowling',
  'book_title': 'Harry Potter and the Prisoner of Azkaban',
  'release_year': 1999},
 {'book_author': 'J.K. Rowling',
  'book_title': 'Harry Potter and the Goblet of Fire',
  'release_year': 2000},
 {'book_author': 'J.K. Rowling',
  'book_title': 'Harry Potter and the Order of the Phoenix',
  'release_year': 2003},
 {'book_author': 'J.K. Rowling',
  'book_title': 'Harry Potter and the Half-Blood Prince',
  'release_year': 2005},
 {'book_author': 'J.K. Rowling',
  'book_title': 'Harry Potter and the Deathly Hallows',
  'release_year': 2007}]


In [5]:
program_iter.job.instruction

'Given a list of book titles, create a list of dictionaries where each dictionary represents a book and includes the title, author, and release year.'

In [9]:
program.job.instruction

"Given the input of a book's title, author, and release year, generate a list of sequels to the book."

In [8]:
from llmp.data_model.job_record import load_engine_from_job

engine = load_engine_from_job(program_iter.job)

print(engine.prompt_builder.build(book_list=output.sequels))

Given a list of book titles, create a list of dictionaries where each dictionary represents a book and includes the title, author, and release year.
# Format instructions

Please return a response in yaml format using the following schema:
```yaml
Book data:
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>
- book_title: <str>
  book_author: <str>
  release_year: <int>

```
Do not include any other information or explanation to your response, 
so that your response can be parsed with yaml.safe_load().
Remember to set the value in quotes using the Double quotation marks 
when values are multiline strings or contain ':'.




{input}


In [6]:
import pprint
pprint.pprint(program_iter.event_log())

[Event(event_id='c944cc828f224390929ed251c21ad381', timestamp='20231107180433', event_type=<EventType.JOB_CREATION: 'job_creation'>, event_metrics=None, job_setting={'instruction': None, 'example_id': []}, job_version=0, example_id=None, example_version=None, extra=None, ref_event_id=None),
 Event(event_id='907ed5df08ed4d45ba6169b8db4a0f2a', timestamp='20231107180445', event_type=<EventType.GENERATION: 'generation'>, event_metrics={'verification_type': 1, 'execution_time': 10.300012111663818, 'token_usage': 988, 'model_name': 'gpt-3.5-turbo', 'model_config': {}, 'failure_rate': 1, 'errors': []}, job_setting=None, job_version=None, example_id=None, example_version=None, extra=None, ref_event_id=None)]


In [5]:
pprint.pprint(program_iter.generation_log())

[{'event_id': '907ed5df08ed4d45ba6169b8db4a0f2a',
  'input': {'book_list': ['Harry Potter and the Chamber of Secrets',
                          'Harry Potter and the Prisoner of Azkaban',
                          'Harry Potter and the Goblet of Fire',
                          'Harry Potter and the Order of the Phoenix',
                          'Harry Potter and the Half-Blood Prince',
                          'Harry Potter and the Deathly Hallows']},
  'output': {'book_data': [{'book_author': 'J.K. Rowling',
                            'book_title': 'Harry Potter and the Chamber of '
                                          'Secrets',
                            'release_year': 1998},
                           {'book_author': 'J.K. Rowling',
                            'book_title': 'Harry Potter and the Prisoner of '
                                          'Azkaban',
                            'release_year': 1999},
                           {'book_author': 'J.K. Rowling',

In [7]:
program_iter.job.idx

'0efcb34c3d7b42ee81fd72a062694640'

In [9]:
from llmp.services.job_storage import JobStorage

JobStorage().load_event_log(program_iter.job)

[Event(event_id='c944cc828f224390929ed251c21ad381', timestamp='20231107180433', event_type=<EventType.JOB_CREATION: 'job_creation'>, event_metrics=None, job_setting={'instruction': None, 'example_id': []}, job_version=0, example_id=None, example_version=None, extra=None, ref_event_id=None),
 Event(event_id='907ed5df08ed4d45ba6169b8db4a0f2a', timestamp='20231107180445', event_type=<EventType.GENERATION: 'generation'>, event_metrics={'verification_type': 1, 'execution_time': 10.300012111663818, 'token_usage': 988, 'model_name': 'gpt-3.5-turbo', 'model_config': {}, 'failure_rate': 1, 'errors': []}, job_setting=None, job_version=None, example_id=None, example_version=None, extra=None, ref_event_id=None)]